# Backpropagation Lab





## 1. (40%) Correctly implement and submit your own code for the backpropagation algorithm. 

## Code requirements 
- Ability to create a network structure with at least one hidden layer and an arbitrary number of nodes.
- Random weight initialization with small random weights with mean of 0 and a variance of 1.
- Use Stochastic/On-line training updates: Iterate and update weights after each training instance (i.e. do not attempt batch updates)
- Implement a validation set based stopping criterion.
- Shuffle training set at each epoch.
- Option to include a momentum term

You may use your own random train/test split or use the scikit-learn version if you want.

Use your Backpropagation algorithm to solve the Debug data. We provide you with several parameters, and you should be able to replicate our results every time. When you are confident it is correct, run your script on the Evaluation data with the same parameters, and print your final weights.

In [11]:
import io
from scipy.io import arff
import urllib
import math
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

class MLP(BaseEstimator,ClassifierMixin):

    def __init__(self,lr=.1, momentum=0, shuffle=True,hidden_layer_widths=None):
        """ Initialize class with chosen hyperparameters.

        Args:
            lr (float): A learning rate / step size.
            shuffle(boolean): Whether to shuffle the training data each epoch. DO NOT SHUFFLE for evaluation / debug datasets.
            momentum(float): The momentum coefficent 
        Optional Args (Args we think will make your life easier):
            hidden_layer_widths (list(int)): A list of integers which defines the width of each hidden layer if hidden layer is none do twice as many hidden nodes as input nodes. (and then one more for the bias node)
            For example: input width 1, then hidden layer will be 3 nodes
        Example:
            mlp = MLP(lr=.2,momentum=.5,shuffle=False,hidden_layer_widths = [3,3]),  <--- this will create a model with two hidden layers, both 3 nodes wide
        """
        self.hidden_layer_widths = hidden_layer_widths
        self.lr = lr
        self.momentum = momentum
        self.shuffle = shuffle
        self.weights = []
        self.prev_change_in_weights = []
        self.output_converter = np.vectorize(_apply_output_single_node)
        self.f_prime_converter = np.vectorize(_get_f_prime_of_output)

        self.runs_left = 0
        self.is_deterministic = False

        self.best_val_score = 0
        self.num_epochs_wo_sig_imp = 0

    def fit(self, X, y, gaussian=True, initial_weights=None, deterministic=0, val_perc=.15):
        """ Fit the data; run the algorithm and adjust the weights to find a good solution

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 2D numpy array with the training targets
        Optional Args (Args we think will make your life easier):
            initial_weights (array-like): allows the user to provide initial weights
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)

        """
        X_train = []
        y_train = []
        X_validate = []
        y_validate = []
        if deterministic != 0:
            self.is_deterministic = True
            self.runs_left = deterministic
            X_train = X
            y_train = y
        else:
            X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=val_perc, random_state=42)

        self.weights = self.initialize_weights(X_train, y_train, gaussian) if not initial_weights else initial_weights
        self._init_prev_change_in_weights()
        X_with_bias = self._add_bias_to_matrix(X_train)

        while not self.stopping_criteria_met(X_validate, y_validate):
            if self.shuffle:
                X_train, y_train = self._shuffle_data(X_train, y_train)
            # Go through all data once
            for data_row in range(0, len(X_with_bias)):
                # Get the current row of data
                cur_inputs = np.array(X_with_bias[data_row])
                cur_targets = np.array(y_train[data_row])

                # Get the outputs (Z) for the whole net
                iteration_outputs = self._get_iteration_outputs(cur_inputs)

                # Get the errors (delta) for the whole net (output and hidden have different formulas)
                output_errors = self._get_all_output_nodes_errors(cur_targets, iteration_outputs[len(iteration_outputs)-1])
                hidden_node_errors = self._get_all_hidden_node_errors(iteration_outputs, output_errors)

                # Update the weights
                all_errors = hidden_node_errors.copy()
                all_errors.append(output_errors)
                inputs_wo_bias = np.array(X_train[data_row])
                all_but_final_output = [inputs_wo_bias]
                all_but_final_output.extend(iteration_outputs[:-1])
                self._update_all_weights(all_errors, all_but_final_output)

        return self

    def stopping_criteria_met(self, X_validate, y_validate):
        if self.is_deterministic:
            self.runs_left -= 1
            if self.runs_left < 0:
                return True
            else:
                return False
        else:
            validation_score = self.score(X_validate, y_validate)
            if validation_score > self.best_val_score:
                self.best_val_score = validation_score
                self.num_epochs_wo_sig_imp = 0
            else:
                self.num_epochs_wo_sig_imp += 1

            if self.num_epochs_wo_sig_imp > 20:
                return False
            else:
                return True



    def predict(self, X):
        """ Predict all classes for a dataset X
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        pass

    def initialize_weights(self, X, y, gaussian):
        """ Initialize weights for perceptron. Don't forget the bias!

        Returns:

        """
        _, num_features = X.shape
        _, num_outputs = y.shape
        if self.hidden_layer_widths == None:
            self.hidden_layer_widths = [num_features * 2]
        weights = []
        input_layer_weights = self._initialize_single_array(num_features + 1, self.hidden_layer_widths[0], gaussian)

        weights.append(input_layer_weights)

        for i in range(0, len(self.hidden_layer_widths)-1):
            cur_layer_weights = self._initialize_single_array(self.hidden_layer_widths[i]+1,self.hidden_layer_widths[i+1], gaussian)
            weights.append(cur_layer_weights)

        output_layer_weights = self._initialize_single_array(self.hidden_layer_widths[len(self.hidden_layer_widths)-1] + 1, num_outputs, gaussian)
        weights.append(output_layer_weights)

        return weights

    def score(self, X, y):
        """ Return accuracy of model on a given dataset. Must implement own score function.

        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 2D numpy array with targets

        Returns:
            score : float
                Mean accuracy of self.predict(X) wrt. y.
        """

        return 0

    def _shuffle_data(self, X, y):
        """ Shuffle the data! This _ prefix suggests that this method should only be called internally.
            It might be easier to concatenate X & y and shuffle a single 2D array, rather than
             shuffling X and y exactly the same way, independently.
        """
        shuffledX, shuffledY = shuffle(X, y)
        return shuffledX, shuffledY

    ### Not required by sk-learn but required by us for grading. Returns the weights.
    def get_weights(self):
        return self.weights

    def _initialize_single_array(self, rows, cols, gaussian):
        if gaussian:
            return np.random.normal(loc=0.0, scale=0.001, size=(rows,cols))
        else:
            return np.zeros((rows, cols))

    def _add_bias_to_matrix(self, npArray):
        _, num_cols = npArray.shape
        return np.insert(npArray, num_cols, 1, axis=1)

    def _get_iteration_outputs(self, inputs_with_bias):
        cur_inputs = inputs_with_bias
        outputs = []
        for i in range(0,len(self.weights)):
            cur_layer_net = cur_inputs.dot(self.weights[i])
            cur_layer_out = self._apply_output_function(cur_layer_net)
            outputs.append(cur_layer_out)
            cur_inputs = np.insert(cur_layer_out, len(cur_layer_out), 1)
        return outputs

    def _apply_output_function(self, net_array):
        return self.output_converter(net_array)

    def _get_all_output_nodes_errors(self, cur_targets, final_outputs):
        num_outputs = final_outputs.shape[0]
        output_errors = []
        for i in range(0, num_outputs):
            output_errors.append(self._get_output_node_error(cur_targets[i], final_outputs[i]))
        return np.array(output_errors)

    def _get_output_node_error(self, target, output):
        return (target - output) * (output) * (1 - output)

    def _get_all_hidden_node_errors(self, all_outputs, output_errors):
        k_errors = output_errors
        hidden_node_errors = []
        for i in range(len(all_outputs)-2, -1, -1):
            j_outputs = all_outputs[i]
            j_k_weights = self.weights[i + 1]
            cur_layer_errors = self._get_hidden_node_layer_error(j_outputs, k_errors, j_k_weights)
            hidden_node_errors.append(cur_layer_errors)
            k_errors = cur_layer_errors
        return hidden_node_errors

    def _get_hidden_node_layer_error(self, j_outputs, k_errors, j_k_weights):
        direct_term = np.dot(j_k_weights, k_errors)
        j_f_primes = self.f_prime_converter(j_outputs)
        cur_layer_errors = []
        for i in range(0, len(j_f_primes)):
            cur_layer_errors.append(j_f_primes[i] * direct_term[i])
        return np.array(cur_layer_errors)

    def _update_all_weights(self, all_node_errors, inh_wob_node_outputs):
        for layer_index in range(0, len(all_node_errors)):
            i_j_weights = self.weights[layer_index]
            j_errors = all_node_errors[layer_index]
            i_outputs = np.insert(inh_wob_node_outputs[layer_index], len(inh_wob_node_outputs[layer_index]), 1)
            prev_i_j_change_weights = self.prev_change_in_weights[layer_index]
            self._update_layer_weights(i_j_weights, j_errors, i_outputs, prev_i_j_change_weights)

    def _update_layer_weights(self, layer_weights, j_errors, i_outputs, prev_i_j_change_weights):
        for i in range(0, len(i_outputs)):
            for j in range(0, len(j_errors)):
                new_change_weight = self.lr * j_errors[j] * i_outputs[i] + self.momentum * prev_i_j_change_weights[i][j]
                layer_weights[i][j] += new_change_weight
                prev_i_j_change_weights[i][j] = new_change_weight


    def _init_prev_change_in_weights(self):
        for i in range(0, len(self.weights)):
            num_rows, num_cols = self.weights[i].shape
            self.prev_change_in_weights.append(np.zeros((num_rows, num_cols)))

def _get_f_prime_of_output(output):
    return output * (1 - output)

def _apply_output_single_node(net):
    output = 1.0 / (1.0 + math.exp(-net))
    return output

def load_data(url: str):
    ftp_stream = urllib.request.urlopen(url)
    data, meta = arff.loadarff(io.StringIO(ftp_stream.read().decode('utf-8')))
    data_frame = pd.DataFrame(data)
    return data_frame

def remove_bytes(num):
    if type(num) == bytes:
        return ord(num) - 48
    return num

byteRemoverFunction = np.vectorize(remove_bytes)

## 1.1 

Debug your model using the following parameters:

Learning Rate = 0.1\
Momentum = 0.5\
Deterministic = 10 [This means run it 10 epochs and should be the same everytime you run it]\
Shuffle = False\
Validation size = 0\
Initial Weights = All zeros\
Hidden Layer Widths = [4]

---

### 1.1.1 Debug 

Debug your model by running it on the [Debug Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/linsep2nonorigin.arff)


Expected Results for Binary Classification (i.e. 1 output node): [debug_bp_0.csv](https://github.com/cs472ta/CS472/blob/master/debug_solutions/debug_bp_0.csv) 

$$ \text{Layer 1} = \begin{bmatrix} -8.81779797\text{e}-05 & -8.81779797\text{e}-05 & -8.81779797\text{e}-05 & -8.81779797\text{e}-05 \\ 7.82757731\text{e}-04 & 7.82757731\text{e}-04 & 7.82757731\text{e}-04 & 7.82757731\text{e}-04 \\ -3.94353645\text{e}-03 & -3.94353645\text{e}-03 & -3.94353645\text{e}-03 & -3.94353645\text{e}-03 \end{bmatrix}$$
                                             
$$ \text{Layer 2} = \begin{bmatrix} -0.01060888 \\ -0.01060888 \\ -0.01060888 \\ -0.01060888 \\ -0.02145495 \end{bmatrix}$$

(The weights do not need to be in this order or shape.)

Expected Results for One Hot Vector Classification (i.e. 2 output nodes): [debug_bp_2outs.csv](https://github.com/cs472ta/CS472/blob/master/debug_solutions/debug_bp_2outs.csv) 

$$ \text{Layer 1} = \begin{bmatrix} -0.00018149 & -0.00018149 & -0.00018149 & -0.00018149 \\ 0.00157468 & 0.00157468 & 0.00157468 & 0.00157468 \\ -0.00788218 & -0.00788218 & -0.00788218 & -0.00788218 \end{bmatrix}$$
                          
$$ \text{Layer 2} = \begin{bmatrix} 0.01050642 & -0.01050642 \\ 0.01050642 & -0.01050642 \\ 0.01050642 & -0.01050642 \\ 0.01050642 & -0.01050642 \\ 0.02148778 & -0.02148778 \end{bmatrix}$$

(The weights do not need to be in this order or shape.)

In [14]:
# Load debug data
debug_data = load_data(r'https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/linsep2nonorigin.arff')
debug_data = byteRemoverFunction(debug_data)
X = debug_data[:,:-1]
y = debug_data[:,-1, None]

# Train on binary debug data
debug_mlp_binary = MLP(momentum=0.5, shuffle=False,hidden_layer_widths=[4])
debug_mlp_binary.fit(X, y, gaussian=False,deterministic=10)

# Print weights for binary
debug_binary_weights = debug_mlp_binary.get_weights()
print('Binary Encoding Weights')
for i in range(0,len(debug_binary_weights)):
    print(f'Layer {i+1}')
    print(debug_binary_weights[i])

# Train on one hot debug data
debug_enc = OneHotEncoder()
y_hot = debug_enc.fit_transform(y).toarray()
debug_mlp_hot = MLP(momentum=0.5, shuffle=False,hidden_layer_widths=[4])
debug_mlp_hot.fit(X, y_hot, gaussian=False,deterministic=10)

# Print weights for one-hot
debug_hot_weights = debug_mlp_hot.get_weights()
print()
print('One-Hot Encoding Weights')
for i in range(0,len(debug_hot_weights)):
    print(f'Layer {i+1}')
    print(debug_hot_weights[i])


Binary Encoding Weights
Layer 1
[[-8.81779797e-05 -8.81779797e-05 -8.81779797e-05 -8.81779797e-05]
 [ 7.82757731e-04  7.82757731e-04  7.82757731e-04  7.82757731e-04]
 [-3.94353645e-03 -3.94353645e-03 -3.94353645e-03 -3.94353645e-03]]
Layer 2
[[-0.01060888]
 [-0.01060888]
 [-0.01060888]
 [-0.01060888]
 [-0.02145495]]

One-Hot Encoding Weights
Layer 1
[[-0.00018149 -0.00018149 -0.00018149 -0.00018149]
 [ 0.00157468  0.00157468  0.00157468  0.00157468]
 [-0.00788218 -0.00788218 -0.00788218 -0.00788218]]
Layer 2
[[ 0.01050642 -0.01050642]
 [ 0.01050642 -0.01050642]
 [ 0.01050642 -0.01050642]
 [ 0.01050642 -0.01050642]
 [ 0.02148778 -0.02148778]]


### 1.1.2 Evaluation

Evaluate your model using the following parameters:

Learning Rate = 0.1\
Momentum = 0.5\
Deterministic = 10 [This means run it 10 epochs and should be the same everytime you run it]\
Shuffle = False\
Validation size = 0\
Initial Weights = All zeros\
Hidden Layer Widths = [4]

We will evaluate your model based on printed weights after training on the [Evaluation Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/data_banknote_authentication.arff)

In [27]:
# Load evaluation data

# Train on evaluation data

# Print weights

## 2. (10%) Backpropagation on the Iris Classification problem.

Load the Iris Dataset [Iris Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/iris.arff)

Parameters:
- One layer of hidden nodes with the number of hidden nodes being twice the number of inputs.
- Use a 80/20 split of the data for the training/test set.
- Use a learning rate of 0.1
- Use a validation set (15% of the training set) taken from the training set for your stopping criteria
- Create one graph with MSE (mean squared error) over epochs from the training set and validation set
- Create one graph with classification accuracy (% classified correctly) over epochs from the training set and validation set
- Print out your test set accuracy

The results for the different measurables should be shown with a different color, line type, etc. Typical backpropagation accuracies for the Iris data set are 85-95%.

---

In [ ]:
# Iris Classification

## 3. (10%) Working with the Vowel Dataset - Learning Rate

Load the Vowel Dataset [Vowel Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/vowel.arff)

- Use one layer of hidden nodes with the number of hidden nodes being twice the number of inputs.
- Use random 80/20 splits of the data for the training/test set.
- Use a validation set (15% of the training set) taken from the training set for your stopping criteria
- Try some different learning rates (LR). Note that each LR will probably require a different number of epochs to learn. 

- For each LR you test, plot their validation's set MSE over Epochs on the same graph. Graph 4-5 different LRs and make them different enough to see a difference between them.

In general, whenever you are testing a parameter such as LR, # of hidden nodes, etc., test values until no more improvement is found. For example, if 20 hidden nodes did better than 10, you would not stop at 20, but would try 40, etc., until you no longer get improvement.

If you would like you may average the results of multiple initial conditions (e.g. 3) per LR, and that obviously would give more accurate results.

<img src=https://raw.githubusercontent.com/cs472ta/CS472/master/images/backpropagation/backprop_val_set_MSE_vs_epochs.png width=500 height=500  align="left">

In [1]:
# Train on each dataset

## 3.1 (5%) Working with the Vowel Dataset - Intuition
- Discuss the effect of varying learning rates. 
- Discuss why the vowel data set might be more difficult than Iris
    - Report both datasets' baseline accuracies and best **test** set accuracies. 
- Consider which of the vowel dataset's given input features you should actually use (Train/test, speaker, gender, ect) and discuss why you chose the ones you did.

Typical backpropagation accuracies for the Vowel data set are above 75%.



*Discuss Intuition here*



## 3.2 (10%) Working with the Vowel Dataset - Hidden Layer Nodes

Using the best LR you discovered, experiment with different numbers of hidden nodes.

- Start with 1 hidden node, then 2, and then double them for each test until you get no more improvement in accuracy. 
- For each number of hidden nodes find the best validation set solution (in terms of validation set MSE).  
- Create one graph with MSE for the training set and validation set on the y-axis and # of hidden nodes on the x-axis.
- Report the final test set accuracy for every # of hidden nodes you experimented on

*Discuss Hidden Layer Nodes here*



## 3.3 (10%) Working with the Vowel Dataset - Momentum

Try some different momentum terms using the best number of hidden nodes and LR from your earlier experiments.

- Create a graph similar to step 3.2, but with momentum on the x-axis and number of epochs until validation set convergence on the y-axis.
- For each momentum term, print the test set accuracy. 
- You are trying to see how much momentum speeds up learning and how it affects accuracy.

*Discuss Momentum here*



## 4.1 (10%) Use the scikit-learn (SK) version of the MLP classifier on the Iris and Vowel data sets.  

You do not need to go through all the steps above, nor graph results. Compare results (accuracy and learning speed) between your version and theirs for some selection of hyper-parameters. Try different hyper-parameters and comment on their effect.

At a minimum, try

- number of hidden nodes and layers
- different activation functions
- learning rate
- regularization and parameters
- momentum (and try nesterov)
- early stopping

In [28]:
# Load sklearn perceptron

# Train on voting dataset

*Record impressions*

## 4.2 (5%) Using the Iris Dataset automatically adjust hyper-parameters using your choice of grid/random search
- Use a grid or random search approach across a reasonable subset of hyper-parameters from the above 
- Report your best accuracy and hyper-parameters. 

In [28]:
# Load sklearn perceptron

# Train on voting dataset

## 5. (Optional 5% Extra credit) For the vowel data set, use the other hyper-parameter approach that you did not use in part 4.2 to find LR, # of hidden nodes, and momentum.  

- Compare and discuss the values found with the ones you found in part 3.


*Discuss findings here*